In [0]:
%sql
drop schema if exists dev_hub.contoso_retail_dw cascade;
drop schema if exists dev_hub.stage_contoso_retail_dw cascade;


In [0]:
checkpoints = {p.path: dbutils.fs.rm(p.path, True) for p in dbutils.fs.ls("/Volumes/dev_hub/checkpoints/contoso_retail_dw")}
checkpoints

In [0]:
%pip install pyyaml

In [0]:
from etl import (
  load_sql, 
  get_environment, 
  Variables, 
  PROJECT
)
import logging
logger = logging.getLogger(f"initialise {PROJECT}")
env = get_environment(spark = spark).name

In [0]:

catalog = f"{env}_landing"
spark.sql(f"USE CATALOG {catalog}")
logger.info(f"default catalog set to {catalog}")

In [0]:
spark.sql(f"create schema if not exists {catalog}.{PROJECT}")

In [0]:
spark.sql(f"""
create external volume if not exists {catalog}.{PROJECT}.{PROJECT}
location 'abfss://landing@datalakegeneva.dfs.core.windows.net/external/{PROJECT}/csv/'
""")

In [0]:

catalog = f"{env}_hub"
spark.sql(f"use catalog {catalog}")
logger.info(f"default catalog set to {catalog}")

In [0]:
spark.sql(f"create schema if not exists {catalog}.{PROJECT}")

In [0]:
spark.sql(f"""
create schema if not exists {catalog}.checkpoints
""")
spark.sql(f"""
create volume if not exists {catalog}.checkpoints.{PROJECT}
""")

In [0]:

variables = {
  Variables.DATABASE : PROJECT
}
sql = load_sql(f"hub/{PROJECT}._audit", variables=variables)
logger.info(f"creating table {PROJECT}._audit")
logger.debug(sql)
spark.sql(sql)